# Blackjack simulator
Modeling and implementing the game of Blackjack is a common pedogogical exercise in Object-Oriented Programming.  It is highly extensible to increasing levels of complexity, from (1) controlling the most basic aspects of gameplay and win-lose decisions between an explicitly programmed dealer and a single human player interacting through a console with no betting or round-to-round awareness ... to (11) providing for rule variations, strategy agents for automated players, risk dispositions for automated players, graphical user interfaces for multiple human players, etc.  Here, I do not plan on "going to 11," but it is important to have a forward-looking picture before deciding how far ahead to actually design for.  This approach might run counter to a common belief that refactoring is always cheaper than designing for change up front, but I believe that only after understanding the real-world domain that you're modeling can you assess the likelihood of changes and, in turn, the costs of designing for change versus later refactoring.

During this exercise, I would like to, at a minimum, simulate *base* Blackjack with one Dealer and multiple players playing multiple rounds with betting, allowing for more than one player strategy option.  The purpose of the simulation will be to demonstrate the probabalistic properties of the game.  Equally important will be to expose the rationale and an analysis/design/development thought process.  A centerpiece of my rationale will be that each design component should have *only one reason to change* and that such **change isolation** will generally lead to code that is cheaper to maintain.  The idea is closely linked to the more common *Single Responsibility Principle*.  However, in contrast to *responsibility*, the concept of *change* is amenable to probabilistic treatment and more easily associated with cost (if A changes, how must we adapt B).

Let's take a look at the domain of the real-world problem more generally.

## High-level analysis of real-world Blackjack
### *Base* game description
The card company *Bicycle* provides an [instructive description](http://www.bicyclecards.com/how-to-play/blackjack/) of the most common version of Blackjack, which we'll consider the *base* game.  [This local copy](file/blackjack_howto_bicycle.pdf) should be read and treated as if it were inline content here.

### Things and actions
*Bicycle*'s description involves several primary real-world "things" and "actions" (alignment of "things" and "actions" in the table has no associative meaning):

Things | Actions
------ | -------
Card<br>Player<br>Hand<br>Dealer<br>Shoe<br>Table | Shuffle<br>Cut<br>Bet<br>Deal<br>Play hand<br>Pay out

Most of the complexity appears in the action "Play hand," which includes the full turn-fulfilling sequence of player or dealer actions.

### Sources of potential change
Variants of the *base* Blackjack involve the same "things" and "actions."  They are differentiated by variations in structure and rules:

(1) ***structural variations*** -- changes in the limitations imposed on the number/composition of cards and players.

(2) ***rule variations*** -- changes in directives and constraints on dealer hand behavior, player options, win-lose conditions, and payout ratios.

The base and variants haver their own win-lose probability landscapes.  Within the landscape, there is a *dynamical* space, a space within which there are options and, in turn,

(3) ***strategy variations*** -- changes that can occur within the scope of one game of a fixed variant and that can, in principle, evolve turn-to-turn independendently.

#### Structural variations
Basic *structural variations* include (a) the number of decks shuffled together for play (number of decks in the shoe), (b) maximum number of players, (c) maximum positions that one player can control, (d) whether and how many non-players can bet on a hand (*back betting*), and (e) periodic versus continuous shuffling.

#### Rule variations
Over 60 *rule variations* are listed on the *Wizard of Odds* website:  https://wizardofodds.com/games/blackjack/rule-variations/.

We need to reduce these variations into a handful of design-significant categories. They appear to directly affect only four aspects of gameplay, with one rule potentially affecting more than one aspect:
a. Dealer hand behavior
b. Player options
c. Win-lose criteria
d. Payout ratios under different conditions

Perhaps the win-lose conditions and payout ratios could considered one category by expanding win-lose decisions to a spectrum of positive and negative payouts.  If so, the impact of rule variants could be absorbed into only three categories:
A. Dealer hand behavior
B. Player options
C. Payout ratios

To me it's not yet obvious which categorization is best, but it might impact our design choices, so I'll just keep it in mind for later.

#### Strategy variations (dynamics)
***Player game strategy*** ranges from superstition through round-isolated decision charts (https://goo.gl/TPYRsV) to accounting for multi-round evolving probability to various levels of precision.  The *Wizard of Odds* site provides a few basic examples of good and bad strategies, and there are many other online examples.

***Player metagame strategy*** is usually non-existant.  Other metagame strategies range from *cheating* to expanding beyond the scope of a single table/game to decide *when* and *where* to play.  For example, players have employed *hard counting* with teams of people watching multiple tables to identify those that are *ripe* for player profit, which actually combines game **and** metagame strategies.  Simultaneously, they would certainly employ techniques to distract from their activity, which would be another metagame strategy.

***Dealer game strategy*** does not need to exist as an independent concept because it is **dictated** by the ruleset and has no room for dynamical variation.  It variation between variants is subsumed by *rule variations*.

***Dealer metagame strategy***, like player strategies, can evolve independently of the ruleset.  One *Stack Exchange* user gives an excellent description of metagame elements that a dealer controls:  https://boardgames.stackexchange.com/a/28261.

## High-level design
The "things" of analysis provide a starting point for coming up with the classes of design.  However, there is a tension between the principles of OO design and the "things" of the real-world.  Otherwise, we would have no semantic gap to bridge!  A good place to start is to consider what each "action" requires in order to act. Does the action's "thing" have access to all of the required information?  Should it?  For example, in the real-world, a Player can observe the history of cards that have been played, apply a strategy, and take action... but from a design perspective that would require access to attributes of almost every object.  What if we refactor the "things" of analysis based on the information needed for each action?  If we redefine a Player to be the mediator between a "thinking" Strategy and the gameplay, then we can let the Strategy have access to everything it needs without exposing it directly to the Player, who only wants a recommendation from the Strategy.  Thinking along those lines, I'll propose a set of Classes.

### Classes

Class | Responsibility
------|---------------
Card | Keep suit, rank, possible values, and visual representation.
Shoe | Shuffle and dispense cards to Hands.
Hand | Calculate its possible scores.
Dealer | Play the dealer hand according to house rules.
Player | Bet and play hand according to strategy.
Strategy | Calculate bets and make play decisions.
Table | Enforce game structure and flow.<br>Coordinate interactions.
OutcomeCalc | Calculate outcomes/payouts.
ProbCalc | Calculate exact probabilities.

### Class Collaborations
* Table creates Shoe, Dealer, Strategies, Players, and Hands.
* Table deals Cards from Shoe to Hands.
* Table associates Players and Dealer with Hands and Strategies.
* Table orchestrates Player/Dealer actions.
* Table collects Bets from Players and associates Bets to Hands.
* Players get Bet recommendations from Strategy.
* Players get Hand action recommendations from Strategy.
* ....
....

#### An Aside for Implementation
At this point, let's mock up a few classes to get a feel for the direction, at a quality level of notes.

In [85]:
import itertools
import functools
import random


class Card:
    val_map = {'A': [1, 11], 'J': [10], 'Q': [10], 'K': [10]}
    val_map.update({str(n): [n] for n in range(2, 11)})

    def __init__(self, rank, suit):
        self._rank = rank
        self._suit = suit
        self._values = Card.val_map[rank]

    @property
    def card(self):
        return self._suit + self._rank

    @property
    def value(self):
        return self._values
    
    def __str__(self):
        (clr0, clr1) = ('\x1b[31m','\x1b[0m') if self._suit in ['\u2665', '\u2666'] else ('', '')
        return clr0 + self.card + clr1


class Shoe:
    def __init__(self, num_decks, depth_threshold):
        self._ndecks = num_decks
        self._shoe_size = num_decks * 52
        self._depth_threshold = depth_threshold
        self._shuffle()
        
    def _shuffle(self):
        ranks = ['A'] + [str(n) for n in range(2, 11)] + ['J', 'Q', 'K']
        suits = ['\u2660', '\u2665', '\u2666', '\u2663']
        decks = [itertools.product(ranks, suits) for ideck in range(self._ndecks)]
        cards = functools.reduce(lambda x, y: x + y,
                                 [list(deck) for deck in decks])

        self._cards = [Card(rank, suit) for (rank, suit) in cards]
        random.shuffle(self._cards)

    def check_depth(self):
        if 1 - len(self._cards)/self._shoe_size >= self._depth_threshold:
            print('RESHUFFLING!')
            self._shuffle()
            
    def deal_one(self):
        if len(self._cards) == 0:
            self.check_depth()
        return self._cards.pop()

    
class Table:
    def __init__(self, num_decks=6, depth_threshold=0.8):
        self._shoe = Shoe(num_decks, depth_threshold)
        self._hands = []
        self._players = []
        self._dealer = Dealer()
    
    def next_card(self):
        return self._shoe.deal_one()

In [218]:
t = Table(1, 0.8)
for j in range(14):
    print('\t'.join([str(t.next_card()) for i in range(4)]))
del t

♦4	♥2	♥10	♣8
♥9	♦8	♦3	♦6
♠K	♠2	♦2	♦J
♣Q	♥5	♣6	♦7
♠J	♦10	♥3	♥Q
♦5	♥J	♥4	♠5
♦Q	♣7	♣K	♣9
♠4	♦A	♠9	♦9
♥7	♥A	♠Q	♠6
♠8	♠3	♣2	♠7
♠A	♣4	♥8	♣J
♥6	♠10	♣3	♦K
♣5	♣A	♣10	♥K
RESHUFFLING!
♠K	♠10	♥9	♦9


In [210]:
import itertools
import functools
import random


class Hand:
    def __init__(self, seat=(0,0), bet=0):
        self._cards = []
        self._hole = None
        self._bet = bet
        self._is_insured = False
        self._seat = seat
        self.__key = hash(self._seat)

    def __hash__(self):
        return self.__key
    
    @property
    def seat(self):
        return self._seat[0]
    @property
    def bet(self):
        return self._bet
    
    @property
    def cards(self):
        return ','.join([card.card for card in self._cards])

    @property
    def scores(self):
        '''Returns all possible scores (more than one if hand contains aces).
        '''
        card_vals = [card.value for card in self._cards]
        sums = [sum(list(vals_list))
                for vals_list in itertools.product(*card_vals)]
        # sums = filter(lambda x: x <= 21, sums)
        return sorted(set(sums))

    @property
    def best_score(self):
        if len(self.scores) == 0:
            return 0
        inplay = [s for s in self.scores if s<=21]
        if len(inplay) == 0:
            return min(self.scores)
        return max([s for s in self.scores if s<=21])
    
    def unhole(self):
        if self._hole is not None:
            self._cards.append(self._hole)
            self._hole = None

    def add_card(self, card, ishole=False):
        if ishole:
            self._hole = card
        else:
            self._cards.append(card)
            
    def __str__(self):
        visible = [str(card) for card in self._cards]
        holes = [] if self._hole is None else ['??']
        cards = visible + holes
        return 'Seat/Split: %d/%d \t %d \t %s' % (self._seat[0],
                                                  self._seat[1],
                                                  self.best_score,
                                                  ' '.join(cards))


class Strategy:
    STAY  = 'S'
    HIT   = 'H'
    UNDEF = 'NA'
    
    def __init__(self, table):
        self._table = table

    def advise_bet(self):
        return 0
    
    def advise_move(self, hand):
        for score in reversed(hand.scores):
            if score > 16:
                return Strategy.STAY
            elif score <= 16:
                return Strategy.HIT
        return Strategy.UNDEF


class Player:
    def __init__(self, seat, strategy):
        self._hands = []
        self._strategy = strategy
        self._seat = seat
    
    @property
    def seat(self):
        return self._seat
    
    def add_hand(self, hand):
        self._hands.append(hand)
        
    def make_bet(self):
        return self._strategy.advise_bet()
        
    def move(self, hand):
        return self._strategy.advise_move(hand)
    
    def clear_hands(self):
        self._hands.clear()
    
    def __str__(self):
        hands_str = '\t\n'.join([str(h) for h in self._hands])
        return 'Player %d:\n\t%s' % (self.seat, hands_str)


class Card:
    val_map = {'A': [1, 11], 'J': [10], 'Q': [10], 'K': [10]}
    val_map.update({str(n): [n] for n in range(2, 11)})

    def __init__(self, rank, suit):
        self._rank = rank
        self._suit = suit
        self._values = Card.val_map[rank]

    @property
    def card(self):
        return self._suit + self._rank

    @property
    def value(self):
        return self._values
    
    def __str__(self):
        (clr0, clr1) = ('\x1b[31m','\x1b[0m') if self._suit in ['\u2665', '\u2666'] else ('', '')
        return clr0 + self.card + clr1


# ########################### #
# A variant can have different valid moves, dealer behavior, payouts, and table setup.
class ValidMoves:
    STAY  = 'S'
    HIT   = 'H'
    
    def __init__(self):
        pass

    def get_options(self, hand_player, hand_dealer):
        valid_options = []
        if min(hand.score<21):
            valid_options.append(ValidMoves.STAY)
            valid_options.append(ValidMoves.HIT)
        return valid_options


class PayOuts:
    def __init__(self):
        pass
    
    def get_payout(self, hand_player, hand_dealer):
        if hand_player.best_score >= hand_dealer.best_score:
            return hand_player.bet

    
class StrategyDealer(Strategy):
    def __init__(self, table):
        Strategy.__init__(self, table)

    def advise_move(self, hand):
        for score in reversed(hand.scores):
            if score > 16:
                return Strategy.STAY
            elif score <= 16:
                return Strategy.HIT
        return Strategy.UNDEF


class Dealer(Hand):
    def __init__(self, strategy):
        Hand.__init__(self)
        self._strategy = strategy

    def move(self):
        return self._strategy.advise_move(self)
    
    def __str__(self):
        return 'DEALER\t%s'%(super(Dealer, self).__str__())


class Shoe:
    def __init__(self, num_decks, depth_threshold):
        self._ndecks = num_decks
        self._shoe_size = num_decks * 52
        self._depth_threshold = depth_threshold
        self._shuffle()
        
    def _shuffle(self):
        ranks = ['A'] + [str(n) for n in range(2, 11)] + ['J', 'Q', 'K']
        suits = ['\u2660', '\u2665', '\u2666', '\u2663']
        decks = [itertools.product(ranks, suits) for ideck in range(self._ndecks)]
        cards = functools.reduce(lambda x, y: x + y,
                                 [list(deck) for deck in decks])

        self._cards = [Card(rank, suit) for (rank, suit) in cards]
        random.shuffle(self._cards)

    def check_depth(self):
        if 1 - len(self._cards)/self._shoe_size >= self._depth_threshold:
            print('RESHUFFLING!')
            self._shuffle()
            
    def deal_one(self):
        if len(self._cards) == 0:
            self.check_depth()
        return self._cards.pop()


class Table:
    def __init__(self, num_decks=6, depth_threshold=0.8):
        self._shoe = Shoe(num_decks, depth_threshold)
        self._hands = []
        self._players = []
        self._validator = ValidMoves()
        self._moves = { 'H': lambda h: h.add_card(self.next_card()) }
    
    def add_player(self, seat, strategy=None):
        assert seat not in [p.seat for p in self._players]
        assert seat > 0
        if strategy is None:
            strategy = Strategy(self)
        self._players.append(Player(seat, strategy))
        
    def next_card(self):
        return self._shoe.deal_one()
        
    def take_bets(self):
        self._hands.clear()
        self._dealer = Dealer(StrategyDealer(self))
        self._hands.append(self._dealer)
        
        for p in self._players:
            p.clear_hands()
            bet = p.make_bet()
            hand = Hand((p.seat, 0), bet)
            p.add_hand(hand)
            self._hands.append(hand)

    def set_hands(self):
        # Technically, the order of dealing does not matter for the purpose
        # strategy testing, but I'll deal in sequence.
        for icard in range(2):
            for h in self._hands[1:]:
                h.add_card(self.next_card())
            is_dealer_hole = icard==0  # first card is down
            self._dealer.add_card(self.next_card(), is_dealer_hole)
        
        for p in self._players:
            print(p)

        print(self._dealer)
        
    def play_players(self):
        for p in self._players:
            # maybe hands and players should be in seat-keyed maps to
            # avoid gymnastics such as this...
            player_hands = [h for h in self._hands if h.seat == p.seat]
            for hand in player_hands:
                mv = p.move(hand)
                while mv != 'S':
                    func = self._moves[mv]
                    func(hand)
                    mv = p.move(hand)
                
            print(p)
        
    def play_dealer(self):
        self._dealer.unhole()
        
        mv = self._dealer.move()
        while mv != 'S':
            func = self._moves[mv]
            func(self._dealer)
            mv = self._dealer.move()
        
        print(self._dealer)
        
    def play_round(self):
        print('*** BETTING ***')
        self.take_bets()
        print('*** DEALING ***')
        self.set_hands()
        print('*** PLAYING ***')
        self.play_players()
        self.play_dealer()
        print('*** PAYING ***')

In [211]:
t = Table()
print(t.next_card())
t.add_player(1)
t.add_player(3)

♦K


In [217]:
t.play_round()

*** BETTING ***
*** DEALING ***
Player 1:
	Seat/Split: 1/0 	 13 	 ♣6 ♣7
Player 3:
	Seat/Split: 3/0 	 21 	 ♥A ♥Q
DEALER	Seat/Split: 0/0 	 4 	 ♦4 ??
*** PLAYING ***
Player 1:
	Seat/Split: 1/0 	 17 	 ♣6 ♣7 ♦4
Player 3:
	Seat/Split: 3/0 	 21 	 ♥A ♥Q
DEALER	Seat/Split: 0/0 	 23 	 ♦4 ♠9 ♦Q
*** PAYING ***
